<a href="https://colab.research.google.com/github/Rahad31/Kl-FedDis-Research-/blob/main/FedAvgnew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
from sklearn.metrics import accuracy_score

# CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Data loading and transformation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Splitting data for 5 clients
def split_data(trainset, num_clients):
    client_len = len(trainset) // num_clients
    client_datasets = random_split(trainset, [client_len] * num_clients)
    return client_datasets

# Initialize clients
def initialize_clients(trainset, num_clients, batch_size=128):
    client_datasets = split_data(trainset, num_clients)
    client_loaders = [DataLoader(client_data, batch_size=batch_size, shuffle=True) for client_data in client_datasets]
    return client_loaders

# FedAvg algorithm: Average the weights from all clients
def fed_avg(global_model, client_models):
    global_dict = global_model.state_dict()
    for key in global_dict.keys():
        global_dict[key] = torch.stack([client_models[i].state_dict()[key].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    return global_model

# Training loop for a client
def train_client(model, trainloader, epochs=1, lr=0.01):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(trainloader):.4f}")

    return model

# Evaluation function
def evaluate(model, testloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    print(f'Test Accuracy: {acc * 100:.2f}%')
    return acc

# Federated Learning with 5 clients, using FedAvg
def federated_learning(trainset, testset, num_clients=5, global_epochs=500, client_epochs=5):
    # Initialize global model and clients
    global_model = CNN()
    client_loaders = initialize_clients(trainset, num_clients)
    testloader = DataLoader(testset, batch_size=128, shuffle=False)

    # Training loop for federated learning
    for global_epoch in range(global_epochs):
        print(f"Global Epoch {global_epoch + 1}/{global_epochs}")

        client_models = []
        for client_idx, client_loader in enumerate(client_loaders):
            client_model = CNN()
            client_model.load_state_dict(global_model.state_dict())  # Load the global model into the client
            print(f"Training on Client {client_idx + 1}")
            client_model = train_client(client_model, client_loader, epochs=client_epochs)
            client_models.append(client_model)

        # Average the weights of client models to update the global model
        global_model = fed_avg(global_model, client_models)

        # Evaluate global model after each global epoch
        evaluate(global_model, testloader)

    print("Training complete.")
    return global_model

# Run the federated learning process
if __name__ == "__main__":
    global_model = federated_learning(trainset, testset, num_clients=5, global_epochs=500, client_epochs=5)


Files already downloaded and verified


KeyboardInterrupt: 

In [ ]:
pip install torch torchvision scikit-learn
python script_name.py



SyntaxError: invalid syntax (<ipython-input-3-f44f316228c2>, line 1)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
from sklearn.metrics import accuracy_score

# CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Data loading and transformation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split data for 5 clients, 80% training, 20% validation
def split_data(trainset, num_clients, val_ratio=0.2):
    client_len = len(trainset) // num_clients
    client_datasets = []
    for _ in range(num_clients):
        client_data, val_data = random_split(trainset, [int(client_len * (1 - val_ratio)), int(client_len * val_ratio)])
        client_datasets.append((client_data, val_data))
    return client_datasets

# Initialize clients with validation data
def initialize_clients(trainset, num_clients, batch_size=128):
    client_datasets = split_data(trainset, num_clients)
    client_loaders = [(DataLoader(client_data, batch_size=batch_size, shuffle=True),
                       DataLoader(val_data, batch_size=batch_size, shuffle=False)) for client_data, val_data in client_datasets]
    return client_loaders

# FedAvg algorithm: Average the weights from all clients
def fed_avg(global_model, client_models):
    global_dict = global_model.state_dict()
    for key in global_dict.keys():
        global_dict[key] = torch.stack([client_models[i].state_dict()[key].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    return global_model

# Training loop for a client
def train_client(model, trainloader, epochs=1, lr=0.01):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    return model

# Validation accuracy function
def validate_client(model, valloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    return acc

# Federated Learning with FedAvg
def federated_learning(trainset, testset, num_clients=5, global_epochs=500, client_epochs=5):
    # Initialize global model and clients
    global_model = CNN()
    client_loaders = initialize_clients(trainset, num_clients)
    testloader = DataLoader(testset, batch_size=128, shuffle=False)

    # Training loop for federated learning
    for global_epoch in range(global_epochs):
        print(f"\nGlobal Epoch {global_epoch + 1}/{global_epochs}")

        client_models = []
        for client_idx, (trainloader, valloader) in enumerate(client_loaders):
            client_model = CNN()
            client_model.load_state_dict(global_model.state_dict())  # Load the global model into the client
            print(f"\nTraining on Client {client_idx + 1}")

            # Train client model
            client_model = train_client(client_model, trainloader, epochs=client_epochs)

            # Validate client model
            val_acc = validate_client(client_model, valloader)

            # Print validation accuracy for Client 5 at round 10 and epoch 10
            if client_idx == 4 and global_epoch + 1 == 10 and client_epochs == 10:
                print(f"\nValidation Accuracy for Client 5 at Round 10, Epoch 10: {val_acc * 100:.2f}%")

            client_models.append(client_model)

        # Average the weights of client models to update the global model
        global_model = fed_avg(global_model, client_models)

    print("Training complete.")
    return global_model

# Run the federated learning process
if __name__ == "__main__":
    global_model = federated_learning(trainset, testset, num_clients=5, global_epochs=500, client_epochs=10)


Files already downloaded and verified
Files already downloaded and verified


ValueError: Sum of input lengths does not equal the length of the input dataset!

In [ ]:
import numpy as np

# Initial dataset provided by the user
FedAvg_accuracy = [
    9.31, 9.48, 9.64, 9.95, 9.98, 10.87, 13.12, 14.64, 15.58, 15.98,
    9.20, 11.76, 13.48, 13.44, 13.01, 12.11, 13.28, 13.38, 13.33, 13.23,
    9.15, 11.68, 14.19, 15.63, 16.14, 16.61, 16.70, 16.66, 16.81, 16.69,
    9.34, 10.89, 13.42, 14.48, 15.96, 16.06, 16.49, 16.93, 17.51, 17.35,
    9.45, 10.70, 11.54, 14.37, 15.37, 16.07, 16.58, 16.80, 17.09, 17.26,
    16.99, 17.40, 16.93, 16.25, 16.00, 16.07, 16.87, 17.79, 18.72, 19.15,
    20.59, 16.87, 16.46, 16.28, 16.40, 16.76, 17.69, 19.04, 20.20, 21.82,
    22.65, 16.93, 16.75, 16.56, 16.44, 16.91, 18.01, 19.42, 20.74, 21.73,
    22.61, 16.91, 16.69, 16.60, 16.46, 16.93, 17.63, 18.58, 20.05, 20.80,
    21.63, 17.02, 16.84, 16.79, 16.96, 17.23, 18.15, 19.26, 20.65, 21.62,
    22.50, 22.37, 22.89, 22.86, 23.78, 24.81, 25.58, 25.56, 25.79, 26.59,
    27.01, 26.74, 23.03, 22.82, 24.33, 24.11, 25.25, 25.84, 26.34, 26.54,
    26.85, 27.61, 23.12, 23.84, 24.45, 24.88, 25.69, 25.91, 25.91, 26.71,
    27.13, 27.32, 23.25, 23.58, 24.44, 24.85, 24.71, 25.79, 25.96, 26.20,
    27.27, 27.64, 23.06, 23.78, 24.24, 24.51, 25.40, 25.79, 26.25, 26.61,
    26.81, 27.65, 27.74, 27.24, 28.80, 28.58, 29.54, 29.22, 30.59, 31.44,
    31.34, 32.81, 33.60, 27.85, 28.44, 29.03, 29.06, 30.40, 30.77, 31.32,
    31.94, 32.58, 33.33, 27.89, 28.23, 29.24, 29.70, 30.34, 30.84, 31.19,
    32.34, 32.11, 32.91, 27.76, 28.76, 28.88, 29.40, 30.29, 30.68, 31.43,
    31.65, 33.27, 32.69, 28.16, 28.51, 29.56, 29.94, 31.07, 30.55, 31.77,
    32.16, 33.40, 33.37, 33.70, 34.14, 35.13, 35.16, 35.40, 36.56, 37.01,
    37.70, 38.11, 38.66, 39.09, 33.81, 34.42, 35.61, 35.89, 36.64, 36.81,
    37.58, 37.13, 38.04, 38.77, 34.17, 34.85, 35.21, 36.24, 36.41, 36.78,
    37.41, 36.94, 38.47, 38.77, 34.27, 34.31, 34.80, 35.84, 35.61, 36.77,
    37.15, 36.87, 38.29, 38.76, 34.29, 34.46, 35.52,35.53, 36.54, 36.73,
    37.42
]

# Method 1: Extrapolation
# Calculate the trend in the last 10 values and use it to extend the series
last_values = np.array(FedAvg_accuracy[-10:])
trend = np.mean(np.diff(last_values))  # Approximate trend based on the last few values
extension_extrapolation = list(last_values + np.arange(1, 101) * trend)

# Method 2: Repetition
extension_repetition = FedAvg_accuracy[:100]  # Repeat the first 100 values

# Combine both extended datasets to make them 500 values long
extended_extrapolation = FedAvg_accuracy + extension_extrapolation[:500-len(FedAvg_accuracy)]
extended_repetition = FedAvg_accuracy + extension_repetition[:500-len(FedAvg_accuracy)]

extended_extrapolation, extended_repetition



ValueError: operands could not be broadcast together with shapes (10,) (100,) 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np

# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data among 5 clients
def split_data(dataset, num_clients=5):
    split_size = len(dataset) // num_clients
    lengths = [split_size] * num_clients
    if len(dataset) % num_clients != 0:
        lengths[-1] += len(dataset) % num_clients
    return random_split(dataset, lengths)

# Create DataLoaders for clients
def create_client_loaders(client_datasets, batch_size=128):
    return [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in client_datasets]

client_datasets = split_data(train_dataset)
client_loaders = create_client_loaders(client_datasets)

# DataLoader for validation (test) set
val_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Train function for one client
def train_client(model, train_loader, criterion, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

# Federated averaging function (FedAvg)
def federated_avg(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.mean(torch.stack([client_model.state_dict()[key].float() for client_model in client_models]), dim=0)
    global_model.load_state_dict(global_state_dict)

# Validation function
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Initialize models for clients and global model
global_model = Net()
client_models = [Net() for _ in range(5)]
criterion = nn.CrossEntropyLoss()

# Training settings
num_rounds = 10  # Total number of communication rounds
num_epochs_per_client = 10  # Number of epochs per client per round
total_epochs = num_rounds * num_epochs_per_client  # 500 total epochs (if num_rounds * num_epochs = 500)

# Training loop
FedAvg_accuracy = []
optimizer_list = [optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9) for client_model in client_models]

for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    # Train each client model on its respective data
    for client_idx, client_loader in enumerate(client_loaders):
        print(f"Training client {client_idx + 1}")
        train_client(client_models[client_idx], client_loader, criterion, optimizer_list[client_idx], epochs=num_epochs_per_client)

    # Perform FedAvg (aggregate models)
    federated_avg(global_model, client_models)

    # Update client models with the aggregated global model
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    # Validate the global model
    val_acc = validate_model(global_model, val_loader)
    FedAvg_accuracy.append(val_acc)
    print(f"Validation Accuracy after round {round_num + 1}: {val_acc:.2f}%")

# After all rounds, print the final trend and validation accuracies

# Method 1: Trend extrapolation based on the last 10 validation accuracies
last_values = np.array(FedAvg_accuracy[-10:])
trend = np.mean(np.diff(last_values))  # Approximate trend based on the last few values
extension_extrapolation = list(last_values[-1] + np.arange(1, 101) * trend)

print("Final validation accuracies over rounds:", FedAvg_accuracy)
print("Extrapolated validation accuracies based on recent trend:", extension_extrapolation)


Files already downloaded and verified
Files already downloaded and verified
Round 1/10
Training client 1
Epoch 1, Loss: 2.3036
Epoch 2, Loss: 2.3025
Epoch 3, Loss: 2.3014
Epoch 4, Loss: 2.3002


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np

# Define the neural network model (Classification Model)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data among 5 clients
def split_data(dataset, num_clients=5):
    split_size = len(dataset) // num_clients
    lengths = [split_size] * num_clients
    if len(dataset) % num_clients != 0:
        lengths[-1] += len(dataset) % num_clients
    return random_split(dataset, lengths)

# Create DataLoaders for clients
def create_client_loaders(client_datasets, batch_size=128):
    return [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in client_datasets]

client_datasets = split_data(train_dataset)
client_loaders = create_client_loaders(client_datasets)

# DataLoader for validation (test) set
val_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Train function for one client
def train_client(model, train_loader, criterion, optimizer, scheduler, epochs=1):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        scheduler.step()

# Federated averaging function (FedAvg)
def federated_avg(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.mean(torch.stack([client_model.state_dict()[key].float() for client_model in client_models]), dim=0)
    global_model.load_state_dict(global_state_dict)

# Validation function
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Initialize models for clients and global model
global_model = Net()
client_models = [Net() for _ in range(5)]
criterion = nn.CrossEntropyLoss()

# Training settings
num_rounds = 10  # Total number of communication rounds
num_epochs_per_client = 10  # Number of epochs per client per round (classification model train = 10)

# Optimizer and scheduler setup (as per Table 2)
optimizer_list = [optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9) for client_model in client_models]
scheduler_list = [optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5) for optimizer in optimizer_list]

# Training loop
FedAvg_accuracy = []

for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    # Train each client model on its respective data
    for client_idx, client_loader in enumerate(client_loaders):
        print(f"Training client {client_idx + 1}")
        train_client(client_models[client_idx], client_loader, criterion, optimizer_list[client_idx], scheduler_list[client_idx], epochs=num_epochs_per_client)

    # Perform FedAvg (aggregate models)
    federated_avg(global_model, client_models)

    # Update client models with the aggregated global model
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    # Validate the global model
    val_acc = validate_model(global_model, val_loader)
    FedAvg_accuracy.append(val_acc)
    print(f"Validation Accuracy after round {round_num + 1}: {val_acc:.2f}%")

# After all rounds, print the final validation accuracies
print("Final validation accuracies over rounds:", FedAvg_accuracy)


Files already downloaded and verified
Files already downloaded and verified
Round 1/10
Training client 1
Training client 2


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np

# Define the neural network model (Classification Model)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data among 5 clients
def split_data(dataset, num_clients=5):
    split_size = len(dataset) // num_clients
    lengths = [split_size] * num_clients
    if len(dataset) % num_clients != 0:
        lengths[-1] += len(dataset) % num_clients
    return random_split(dataset, lengths)

# Create DataLoaders for clients
def create_client_loaders(client_datasets, batch_size=128):
    return [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in client_datasets]

client_datasets = split_data(train_dataset)
client_loaders = create_client_loaders(client_datasets)

# DataLoader for validation (test) set
val_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Train function for one client
def train_client(model, train_loader, criterion, optimizer, scheduler, val_loader, epochs=1):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        scheduler.step()
        # Validate the model after every epoch
        val_acc = validate_model(model, val_loader)
        print(f"Epoch {epoch + 1} - Validation Accuracy: {val_acc:.2f}%")

# Federated averaging function (FedAvg)
def federated_avg(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.mean(torch.stack([client_model.state_dict()[key].float() for client_model in client_models]), dim=0)
    global_model.load_state_dict(global_state_dict)

# Validation function
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Initialize models for clients and global model
global_model = Net()
client_models = [Net() for _ in range(5)]
criterion = nn.CrossEntropyLoss()

# Training settings
num_rounds = 10  # Total number of communication rounds
num_epochs_per_client = 10  # Number of epochs per client per round (classification model train = 10)

# Optimizer and scheduler setup (as per Table 2)
optimizer_list = [optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9) for client_model in client_models]
scheduler_list = [optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5) for optimizer in optimizer_list]

# Training loop
FedAvg_accuracy = []

for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    # Train each client model on its respective data and print validation accuracy after each epoch
    for client_idx, client_loader in enumerate(client_loaders):
        print(f"Training client {client_idx + 1}")
        train_client(client_models[client_idx], client_loader, criterion, optimizer_list[client_idx], scheduler_list[client_idx], val_loader, epochs=num_epochs_per_client)

    # Perform FedAvg (aggregate models)
    federated_avg(global_model, client_models)

    # Update client models with the aggregated global model
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    # Validate the global model after each round
    val_acc = validate_model(global_model, val_loader)
    FedAvg_accuracy.append(val_acc)
    print(f"Validation Accuracy after round {round_num + 1}: {val_acc:.2f}%")

# After all rounds, print the final validation accuracies
print("Final validation accuracies over rounds:", FedAvg_accuracy)


Files already downloaded and verified
Files already downloaded and verified
Round 1/10
Training client 1
Epoch 1 - Validation Accuracy: 10.02%
Epoch 2 - Validation Accuracy: 10.02%
Epoch 3 - Validation Accuracy: 10.05%
Epoch 4 - Validation Accuracy: 10.05%
Epoch 5 - Validation Accuracy: 10.12%
Epoch 6 - Validation Accuracy: 10.30%
Epoch 7 - Validation Accuracy: 10.33%
Epoch 8 - Validation Accuracy: 11.05%
Epoch 9 - Validation Accuracy: 12.98%
Epoch 10 - Validation Accuracy: 15.28%
Training client 2
Epoch 1 - Validation Accuracy: 9.22%
Epoch 2 - Validation Accuracy: 10.07%
Epoch 3 - Validation Accuracy: 11.21%
Epoch 4 - Validation Accuracy: 12.15%
Epoch 5 - Validation Accuracy: 14.53%
Epoch 6 - Validation Accuracy: 17.01%
Epoch 7 - Validation Accuracy: 18.21%
Epoch 8 - Validation Accuracy: 19.22%
Epoch 9 - Validation Accuracy: 20.35%
Epoch 10 - Validation Accuracy: 21.51%
Training client 3
Epoch 1 - Validation Accuracy: 10.15%
Epoch 2 - Validation Accuracy: 10.24%
Epoch 3 - Validation A

In [ ]:
%%time

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np

# Define the neural network model (Classification Model)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data among 5 clients
def split_data(dataset, num_clients=5):
    split_size = len(dataset) // num_clients
    lengths = [split_size] * num_clients
    if len(dataset) % num_clients != 0:
        lengths[-1] += len(dataset) % num_clients
    return random_split(dataset, lengths)

# Create DataLoaders for clients
def create_client_loaders(client_datasets, batch_size=128):
    return [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in client_datasets]

client_datasets = split_data(train_dataset)
client_loaders = create_client_loaders(client_datasets)

# DataLoader for validation (test) set
val_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Train function for one client
def train_client(model, train_loader, criterion, optimizer, scheduler, val_loader, epochs=1):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        scheduler.step()
        # Validate the model after every epoch
        val_acc = validate_model(model, val_loader)
        print(f"Epoch {epoch + 1} - Validation Accuracy: {val_acc:.2f}%")

# Federated averaging function (FedAvg)
def federated_avg(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.mean(torch.stack([client_model.state_dict()[key].float() for client_model in client_models]), dim=0)
    global_model.load_state_dict(global_state_dict)

# Validation function
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Initialize models for clients and global model
global_model = Net()
client_models = [Net() for _ in range(5)]
criterion = nn.CrossEntropyLoss()

# Training settings
num_rounds = 10  # Total number of communication rounds
num_epochs_per_client = 10  # Number of epochs per client per round (classification model train = 10)

# Optimizer and scheduler setup (as per Table 2)
optimizer_list = [optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9) for client_model in client_models]
scheduler_list = [optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5) for optimizer in optimizer_list]

# Training loop
FedAvg_accuracy = []

for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    # Train each client model on its respective data and print validation accuracy after each epoch
    for client_idx, client_loader in enumerate(client_loaders):
        print(f"Training client {client_idx + 1}")
        train_client(client_models[client_idx], client_loader, criterion, optimizer_list[client_idx], scheduler_list[client_idx], val_loader, epochs=num_epochs_per_client)

    # Perform FedAvg (aggregate models)
    federated_avg(global_model, client_models)

    # Update client models with the aggregated global model
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    # Validate the global model after each round
    val_acc = validate_model(global_model, val_loader)
    FedAvg_accuracy.append(val_acc)
    print(f"Validation Accuracy after round {round_num + 1}: {val_acc:.2f}%")

# After all rounds, print the final validation accuracies
print("Final validation accuracies over rounds:", FedAvg_accuracy)


100%|██████████| 170M/170M [00:03<00:00, 53.3MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Round 1/10
Training client 1
Epoch 1 - Validation Accuracy: 9.72%
Epoch 2 - Validation Accuracy: 9.34%
Epoch 3 - Validation Accuracy: 9.19%
Epoch 4 - Validation Accuracy: 9.79%
Epoch 5 - Validation Accuracy: 9.94%
Epoch 6 - Validation Accuracy: 10.01%
Epoch 7 - Validation Accuracy: 10.06%
Epoch 8 - Validation Accuracy: 9.99%
Epoch 9 - Validation Accuracy: 10.04%
Epoch 10 - Validation Accuracy: 10.61%
Training client 2
Epoch 1 - Validation Accuracy: 9.38%
Epoch 2 - Validation Accuracy: 9.84%
Epoch 3 - Validation Accuracy: 12.00%
Epoch 4 - Validation Accuracy: 12.25%
Epoch 5 - Validation Accuracy: 12.53%
Epoch 6 - Validation Accuracy: 12.55%
Epoch 7 - Validation Accuracy: 13.08%
Epoch 8 - Validation Accuracy: 14.38%
Epoch 9 - Validation Accuracy: 16.26%
Epoch 10 - Validation Accuracy: 16.64%
Training client 3
Epoch 1 - Validation Accuracy: 10.00%
Epoch 2 - Validation Accuracy: 10.00%
Epoch 3 - Valida

In [ ]:
%%time

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np

# Define the neural network model (Classification Model)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data among 5 clients
def split_data(dataset, num_clients=5):
    split_size = len(dataset) // num_clients
    lengths = [split_size] * num_clients
    if len(dataset) % num_clients != 0:
        lengths[-1] += len(dataset) % num_clients
    return random_split(dataset, lengths)

# Create DataLoaders for clients
def create_client_loaders(client_datasets, batch_size=128):
    return [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in client_datasets]

client_datasets = split_data(train_dataset)
client_loaders = create_client_loaders(client_datasets)

# DataLoader for validation (test) set
val_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Train function for one client
def train_client(model, train_loader, criterion, optimizer, scheduler, val_loader, epochs=1):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        scheduler.step()
        # Validate the model after every epoch
        val_acc = validate_model(model, val_loader)
        print(f"Epoch {epoch + 1} - Validation Accuracy: {val_acc:.2f}%")

# Federated averaging function (FedAvg)
def federated_avg(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.mean(torch.stack([client_model.state_dict()[key].float() for client_model in client_models]), dim=0)
    global_model.load_state_dict(global_state_dict)

# Validation function
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Initialize models for clients and global model
global_model = Net()
client_models = [Net() for _ in range(5)]
criterion = nn.CrossEntropyLoss()

# Training settings
num_rounds = 10  # Total number of communication rounds
num_epochs_per_client = 5  # Number of epochs per client per round (classification model train = 10)

# Optimizer and scheduler setup (as per Table 2)
optimizer_list = [optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9) for client_model in client_models]
scheduler_list = [optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5) for optimizer in optimizer_list]

# Training loop
FedAvg_accuracy = []

for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    # Train each client model on its respective data and print validation accuracy after each epoch
    for client_idx, client_loader in enumerate(client_loaders):
        print(f"Training client {client_idx + 1}")
        train_client(client_models[client_idx], client_loader, criterion, optimizer_list[client_idx], scheduler_list[client_idx], val_loader, epochs=num_epochs_per_client)

    # Perform FedAvg (aggregate models)
    federated_avg(global_model, client_models)

    # Update client models with the aggregated global model
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())

    # Validate the global model after each round
    val_acc = validate_model(global_model, val_loader)
    FedAvg_accuracy.append(val_acc)
    print(f"Validation Accuracy after round {round_num + 1}: {val_acc:.2f}%")

# After all rounds, print the final validation accuracies
print("Final validation accuracies over rounds:", FedAvg_accuracy)


Files already downloaded and verified
Files already downloaded and verified
Round 1/10
Training client 1
Epoch 1 - Validation Accuracy: 9.90%
Epoch 2 - Validation Accuracy: 10.10%
Epoch 3 - Validation Accuracy: 10.18%
Epoch 4 - Validation Accuracy: 10.55%
Epoch 5 - Validation Accuracy: 12.33%
Training client 2
Epoch 1 - Validation Accuracy: 10.03%
Epoch 2 - Validation Accuracy: 10.03%
Epoch 3 - Validation Accuracy: 10.29%
Epoch 4 - Validation Accuracy: 10.77%
Epoch 5 - Validation Accuracy: 11.97%
Training client 3
Epoch 1 - Validation Accuracy: 10.01%
Epoch 2 - Validation Accuracy: 10.19%
Epoch 3 - Validation Accuracy: 10.81%
Epoch 4 - Validation Accuracy: 11.85%
Epoch 5 - Validation Accuracy: 13.15%
Training client 4
Epoch 1 - Validation Accuracy: 10.64%
Epoch 2 - Validation Accuracy: 11.81%
Epoch 3 - Validation Accuracy: 13.70%
Epoch 4 - Validation Accuracy: 15.14%
Epoch 5 - Validation Accuracy: 15.34%
Training client 5
Epoch 1 - Validation Accuracy: 13.04%
Epoch 2 - Validation Accur

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.datasets import CIFAR10
from typing import Dict
import random
import numpy as np

# Define CNN Classification Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load CIFAR-10 dataset
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # Fixed error here
    ]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Split dataset for clients
def initialize_clients(dataset, num_clients=5):
    data_size = len(dataset) // num_clients
    client_datasets = [
        Subset(dataset, list(range(i * data_size, (i + 1) * data_size)))
        for i in range(num_clients)
    ]
    return client_datasets

# Initialize clients
num_clients = 5
client_datasets = initialize_clients(full_dataset, num_clients=num_clients)
client_loaders = {
    f"client_{i}": DataLoader(client_datasets[i], batch_size=128, shuffle=True)
    for i in range(num_clients)
}

# Define training function
def train_model(net, dataloader, epochs, lr=0.001, momentum=0.9, step_size=20, gamma=0.5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()

# Aggregate models
def average_weights(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.stack(
            [client_model.state_dict()[key] for client_model in client_models]
        ).mean(0)
    global_model.load_state_dict(global_state_dict)

# Evaluate the model
def evaluate_model(net, dataloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Federated training procedure
def federated_training(
    num_rounds=10, num_epochs=10, lr=0.001, momentum=0.9, step_size=20, gamma=0.5
):
    global_model = Net()
    client_models = [Net() for _ in range(num_clients)]
    test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

    for rnd in range(num_rounds):
        print(f"Round {rnd + 1}/{num_rounds}")
        for i, client_loader in enumerate(client_loaders.values()):
            client_models[i].load_state_dict(global_model.state_dict())
            train_model(client_models[i], client_loader, epochs=num_epochs, lr=lr, momentum=momentum, step_size=step_size, gamma=gamma)

        # Average weights to update global model
        average_weights(global_model, client_models)

        # Evaluate the global model
        accuracy = evaluate_model(global_model, test_loader)
        print(f"Test Accuracy after round {rnd + 1}: {accuracy:.2f}%")

    # Save the final model
    torch.save(global_model.state_dict(), "fedavg_cifar10.pth")

# Run federated training
if __name__ == "__main__":
    federated_training()


100%|██████████| 170498071/170498071 [00:02<00:00, 84190379.08it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Round 1/10
Test Accuracy after round 1: 12.24%
Round 2/10
Test Accuracy after round 2: 23.14%
Round 3/10
Test Accuracy after round 3: 30.56%
Round 4/10
Test Accuracy after round 4: 38.06%
Round 5/10
Test Accuracy after round 5: 42.20%
Round 6/10
Test Accuracy after round 6: 45.16%
Round 7/10
Test Accuracy after round 7: 47.80%
Round 8/10
Test Accuracy after round 8: 49.74%
Round 9/10
Test Accuracy after round 9: 51.04%
Round 10/10
Test Accuracy after round 10: 51.85%


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
import numpy as np
import random

# Define CIFAR-10 dataset and transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_set, val_set = random_split(train_set, [train_size, val_size])

# Define DataLoader
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)

# Define the classification model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define training function with validation accuracy
def train_model(
    net, train_loader, val_loader, epochs, lr=0.001, momentum=0.9, step_size=20, gamma=0.5
):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    for epoch in range(epochs):
        # Training phase
        net.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()
        epoch_loss = running_loss / len(train_loader)

        # Validation phase
        net.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = net(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total

        # Print results for the epoch
        print(
            f"Epoch [{epoch + 1}/{epochs}], Training Loss: {epoch_loss:.3f}, Validation Accuracy: {val_accuracy:.2f}%"
        )

# Define function to average weights during federated training
def average_weights(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.stack(
            [client_models[i].state_dict()[key] for i in range(len(client_models))]
        ).mean(dim=0)
    global_model.load_state_dict(global_state_dict)

# Define evaluation function
def evaluate_model(net, test_loader):
    net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Define function to initialize clients
def initialize_clients(num_clients, train_set):
    client_loaders = {}
    dataset_size = len(train_set)
    indices = list(range(dataset_size))
    random.shuffle(indices)

    split_size = dataset_size // num_clients
    for i in range(num_clients):
        client_indices = indices[i * split_size: (i + 1) * split_size]
        client_subset = torch.utils.data.Subset(train_set, client_indices)
        client_loaders[f"client_{i}"] = DataLoader(client_subset, batch_size=128, shuffle=True)
    return client_loaders

# Federated training procedure
def federated_training(
    num_rounds=10, num_epochs=10, num_clients=5, lr=0.001, momentum=0.9, step_size=20, gamma=0.5
):
    global_model = Net()
    client_models = [Net() for _ in range(num_clients)]
    client_loaders = initialize_clients(num_clients, train_set)

    for rnd in range(num_rounds):
        print(f"Round {rnd + 1}/{num_rounds}")
        for i, client_loader in enumerate(client_loaders.values()):
            print(f"Training Client {i + 1}")
            client_models[i].load_state_dict(global_model.state_dict())
            train_model(
                client_models[i],
                client_loader,
                val_loader,
                epochs=num_epochs,
                lr=lr,
                momentum=momentum,
                step_size=step_size,
                gamma=gamma,
            )

        # Average weights to update global model
        average_weights(global_model, client_models)

        # Evaluate the global model
        accuracy = evaluate_model(global_model, test_loader)
        print(f"Round {rnd + 1} Complete: Test Accuracy: {accuracy:.2f}%\n")

    # Save the final model
    torch.save(global_model.state_dict(), "fedavg_cifar10.pth")

# Run federated training
if __name__ == "__main__":
    federated_training()


100%|██████████| 170M/170M [00:02<00:00, 75.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Round 1/10
Training Client 1
Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 9.68%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 9.69%
Epoch [3/10], Training Loss: 2.303, Validation Accuracy: 9.69%
Epoch [4/10], Training Loss: 2.302, Validation Accuracy: 9.67%
Epoch [5/10], Training Loss: 2.302, Validation Accuracy: 9.69%
Epoch [6/10], Training Loss: 2.301, Validation Accuracy: 9.71%
Epoch [7/10], Training Loss: 2.301, Validation Accuracy: 9.76%
Epoch [8/10], Training Loss: 2.300, Validation Accuracy: 9.92%
Epoch [9/10], Training Loss: 2.299, Validation Accuracy: 9.97%
Epoch [10/10], Training Loss: 2.298, Validation Accuracy: 10.59%
Training Client 2
Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 9.69%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 9.69%


KeyboardInterrupt: 

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader, random_split
import random

# Define CIFAR-10 dataset and transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define DataLoader for test dataset
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)

# Define the classification model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define training function for a single client
def train_client_model(net, data_loader, val_loader, epochs, lr=0.001, momentum=0.9, step_size=20, gamma=0.5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    for epoch in range(epochs):
        # Training phase
        net.train()
        running_loss = 0.0
        for inputs, labels in data_loader:
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()
        epoch_loss = running_loss / len(data_loader)

        # Validation phase
        net.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = net(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{epochs}], Training Loss: {epoch_loss:.3f}, Validation Accuracy: {val_accuracy:.2f}%")
    return net

# Define function to average weights during federated training
def average_weights(global_model, client_models):
    global_state_dict = global_model.state_dict()
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.stack(
            [client_models[i].state_dict()[key] for i in range(len(client_models))]
        ).mean(dim=0)
    global_model.load_state_dict(global_state_dict)

# Define evaluation function
def evaluate_model(net, test_loader):
    net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Define function to initialize clients
def initialize_clients(num_clients, train_set):
    client_loaders = {}
    dataset_size = len(train_set)
    indices = list(range(dataset_size))
    random.shuffle(indices)

    split_size = dataset_size // num_clients
    for i in range(num_clients):
        client_indices = indices[i * split_size: (i + 1) * split_size]
        client_subset = torch.utils.data.Subset(train_set, client_indices)
        client_loaders[f"client_{i}"] = DataLoader(client_subset, batch_size=128, shuffle=True)
    return client_loaders

# Federated training procedure
def federated_training(
    num_rounds=10, num_epochs=10, num_clients=5, lr=0.001, momentum=0.9, step_size=20, gamma=0.5
):
    global_model = Net()
    client_models = [Net() for _ in range(num_clients)]

    # Split train_set into 80% training and 20% validation
    train_size = int(0.8 * len(train_set))
    val_size = len(train_set) - train_size
    train_subset, val_subset = random_split(train_set, [train_size, val_size])

    # Initialize client-specific DataLoaders
    client_loaders = initialize_clients(num_clients, train_subset)
    val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)

    for rnd in range(num_rounds):
        print(f"--- Round {rnd + 1}/{num_rounds} ---")
        for i, client_loader in enumerate(client_loaders.values()):
            print(f"Client {i + 1}:")
            client_models[i].load_state_dict(global_model.state_dict())  # Load global model
            train_client_model(
                client_models[i],
                client_loader,
                val_loader,
                epochs=num_epochs,
                lr=lr,
                momentum=momentum,
                step_size=step_size,
                gamma=gamma,
            )

        # Average weights to update global model
        average_weights(global_model, client_models)

        # Evaluate the global model
        accuracy = evaluate_model(global_model, test_loader)
        print(f"--- Round {rnd + 1} Complete: Global Test Accuracy: {accuracy:.2f}% ---\n")

    # Save the final model
    torch.save(global_model.state_dict(), "fedavg_cifar10.pth")

# Run federated training
if __name__ == "__main__":
    federated_training()


Files already downloaded and verified
Files already downloaded and verified
--- Round 1/10 ---
Client 1:
Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 11.41%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 10.10%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 9.96%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 9.85%
Epoch [5/10], Training Loss: 2.303, Validation Accuracy: 9.85%
Epoch [6/10], Training Loss: 2.302, Validation Accuracy: 9.82%
Epoch [7/10], Training Loss: 2.302, Validation Accuracy: 9.83%
Epoch [8/10], Training Loss: 2.301, Validation Accuracy: 9.83%
Epoch [9/10], Training Loss: 2.301, Validation Accuracy: 9.85%
Epoch [10/10], Training Loss: 2.301, Validation Accuracy: 9.89%
Client 2:
Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 11.49%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 11.37%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 11.23%
Epoch [4/10], Training Loss: 2.304, Validation Accuracy: 11.

In [ ]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 11.41%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 10.10%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 9.96%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 9.85%
Epoch [5/10], Training Loss: 2.303, Validation Accuracy: 9.85%
Epoch [6/10], Training Loss: 2.302, Validation Accuracy: 9.82%
Epoch [7/10], Training Loss: 2.302, Validation Accuracy: 9.83%
Epoch [8/10], Training Loss: 2.301, Validation Accuracy: 9.83%
Epoch [9/10], Training Loss: 2.301, Validation Accuracy: 9.85%
Epoch [10/10], Training Loss: 2.301, Validation Accuracy: 9.89%
Client 2:
Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 11.49%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 11.37%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 11.23%
Epoch [4/10], Training Loss: 2.304, Validation Accuracy: 11.50%
Epoch [5/10], Training Loss: 2.303, Validation Accuracy: 12.31%
Epoch [6/10], Training Loss: 2.303, Validation Accuracy: 11.58%
Epoch [7/10], Training Loss: 2.302, Validation Accuracy: 10.71%
Epoch [8/10], Training Loss: 2.302, Validation Accuracy: 10.07%
Epoch [9/10], Training Loss: 2.302, Validation Accuracy: 10.00%
Epoch [10/10], Training Loss: 2.301, Validation Accuracy: 9.93%
Client 3:
Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 11.48%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 10.44%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 10.58%
Epoch [4/10], Training Loss: 2.304, Validation Accuracy: 10.39%
Epoch [5/10], Training Loss: 2.303, Validation Accuracy: 10.28%
Epoch [6/10], Training Loss: 2.303, Validation Accuracy: 10.47%
Epoch [7/10], Training Loss: 2.302, Validation Accuracy: 11.27%
Epoch [8/10], Training Loss: 2.302, Validation Accuracy: 11.55%
Epoch [9/10], Training Loss: 2.301, Validation Accuracy: 12.25%
Epoch [10/10], Training Loss: 2.301, Validation Accuracy: 12.64%
Client 4:
Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 10.39%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 9.95%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 9.90%
Epoch [4/10], Training Loss: 2.304, Validation Accuracy: 9.89%
Epoch [5/10], Training Loss: 2.303, Validation Accuracy: 9.87%
Epoch [6/10], Training Loss: 2.303, Validation Accuracy: 9.87%
Epoch [7/10], Training Loss: 2.302, Validation Accuracy: 9.87%
Epoch [8/10], Training Loss: 2.302, Validation Accuracy: 9.87%
Epoch [9/10], Training Loss: 2.301, Validation Accuracy: 9.87%
Epoch [10/10], Training Loss: 2.301, Validation Accuracy: 9.87%
Client 5:
Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 10.89%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 10.67%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 9.91%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 9.85%
Epoch [5/10], Training Loss: 2.303, Validation Accuracy: 9.84%
Epoch [6/10], Training Loss: 2.302, Validation Accuracy: 9.84%
Epoch [7/10], Training Loss: 2.302, Validation Accuracy: 9.84%
Epoch [8/10], Training Loss: 2.302, Validation Accuracy: 9.84%
Epoch [9/10], Training Loss: 2.301, Validation Accuracy: 9.85%
Epoch [10/10], Training Loss: 2.301, Validation Accuracy: 9.85%
--- Round 1 Complete: Global Test Accuracy: 12.82% ---

--- Round 2/10 ---
Client 1:
Epoch [1/10], Training Loss: 2.301, Validation Accuracy: 12.64%
Epoch [2/10], Training Loss: 2.300, Validation Accuracy: 12.62%
Epoch [3/10], Training Loss: 2.300, Validation Accuracy: 11.74%
Epoch [4/10], Training Loss: 2.299, Validation Accuracy: 12.25%
Epoch [5/10], Training Loss: 2.299, Validation Accuracy: 12.77%
Epoch [6/10], Training Loss: 2.298, Validation Accuracy: 13.15%
Epoch [7/10], Training Loss: 2.297, Validation Accuracy: 13.48%
Epoch [8/10], Training Loss: 2.296, Validation Accuracy: 14.91%
Epoch [9/10], Training Loss: 2.294, Validation Accuracy: 15.73%
Epoch [10/10], Training Loss: 2.293, Validation Accuracy: 16.03%
Client 2:
Epoch [1/10], Training Loss: 2.301, Validation Accuracy: 13.97%
Epoch [2/10], Training Loss: 2.301, Validation Accuracy: 14.24%
Epoch [3/10], Training Loss: 2.300, Validation Accuracy: 12.69%
Epoch [4/10], Training Loss: 2.300, Validation Accuracy: 12.68%
Epoch [5/10], Training Loss: 2.299, Validation Accuracy: 12.96%
Epoch [6/10], Training Loss: 2.298, Validation Accuracy: 13.94%
Epoch [7/10], Training Loss: 2.298, Validation Accuracy: 14.58%
Epoch [8/10], Training Loss: 2.297, Validation Accuracy: 14.44%
Epoch [9/10], Training Loss: 2.296, Validation Accuracy: 15.77%
Epoch [10/10], Training Loss: 2.294, Validation Accuracy: 16.67%
Client 3:
Epoch [1/10], Training Loss: 2.301, Validation Accuracy: 13.78%
Epoch [2/10], Training Loss: 2.301, Validation Accuracy: 13.68%
Epoch [3/10], Training Loss: 2.300, Validation Accuracy: 13.63%
Epoch [4/10], Training Loss: 2.299, Validation Accuracy: 14.11%
Epoch [5/10], Training Loss: 2.299, Validation Accuracy: 14.08%
Epoch [6/10], Training Loss: 2.298, Validation Accuracy: 14.45%
Epoch [7/10], Training Loss: 2.297, Validation Accuracy: 14.52%
Epoch [8/10], Training Loss: 2.296, Validation Accuracy: 14.65%
Epoch [9/10], Training Loss: 2.295, Validation Accuracy: 15.43%
Epoch [10/10], Training Loss: 2.293, Validation Accuracy: 16.25%
Client 4:
Epoch [1/10], Training Loss: 2.301, Validation Accuracy: 12.48%
Epoch [2/10], Training Loss: 2.301, Validation Accuracy: 10.35%
Epoch [3/10], Training Loss: 2.300, Validation Accuracy: 9.97%
Epoch [4/10], Training Loss: 2.300, Validation Accuracy: 9.96%
Epoch [5/10], Training Loss: 2.299, Validation Accuracy: 9.94%
Epoch [6/10], Training Loss: 2.298, Validation Accuracy: 9.98%
Epoch [7/10], Training Loss: 2.297, Validation Accuracy: 10.27%
Epoch [8/10], Training Loss: 2.297, Validation Accuracy: 10.51%
Epoch [9/10], Training Loss: 2.295, Validation Accuracy: 11.47%
Epoch [10/10], Training Loss: 2.294, Validation Accuracy: 12.15%
Client 5:
Epoch [1/10], Training Loss: 2.301, Validation Accuracy: 13.08%
Epoch [2/10], Training Loss: 2.300, Validation Accuracy: 12.04%
Epoch [3/10], Training Loss: 2.300, Validation Accuracy: 10.53%
Epoch [4/10], Training Loss: 2.299, Validation Accuracy: 10.09%
Epoch [5/10], Training Loss: 2.299, Validation Accuracy: 10.01%
Epoch [6/10], Training Loss: 2.298, Validation Accuracy: 10.10%
Epoch [7/10], Training Loss: 2.297, Validation Accuracy: 10.71%
Epoch [8/10], Training Loss: 2.296, Validation Accuracy: 11.71%
Epoch [9/10], Training Loss: 2.295, Validation Accuracy: 13.14%
Epoch [10/10], Training Loss: 2.294, Validation Accuracy: 13.79%
--- Round 2 Complete: Global Test Accuracy: 18.27% ---

--- Round 3/10 ---
Client 1:
Epoch [1/10], Training Loss: 2.292, Validation Accuracy: 19.21%
Epoch [2/10], Training Loss: 2.289, Validation Accuracy: 19.31%
Epoch [3/10], Training Loss: 2.285, Validation Accuracy: 19.58%
Epoch [4/10], Training Loss: 2.280, Validation Accuracy: 19.79%
Epoch [5/10], Training Loss: 2.271, Validation Accuracy: 19.42%
Epoch [6/10], Training Loss: 2.258, Validation Accuracy: 19.33%
Epoch [7/10], Training Loss: 2.237, Validation Accuracy: 19.40%
Epoch [8/10], Training Loss: 2.208, Validation Accuracy: 20.73%
Epoch [9/10], Training Loss: 2.174, Validation Accuracy: 22.36%
Epoch [10/10], Training Loss: 2.139, Validation Accuracy: 23.26%
Client 2:
Epoch [1/10], Training Loss: 2.293, Validation Accuracy: 20.07%
Epoch [2/10], Training Loss: 2.290, Validation Accuracy: 20.06%
Epoch [3/10], Training Loss: 2.287, Validation Accuracy: 19.94%
Epoch [4/10], Training Loss: 2.282, Validation Accuracy: 20.12%
Epoch [5/10], Training Loss: 2.275, Validation Accuracy: 19.60%
Epoch [6/10], Training Loss: 2.263, Validation Accuracy: 19.73%
Epoch [7/10], Training Loss: 2.245, Validation Accuracy: 20.21%
Epoch [8/10], Training Loss: 2.220, Validation Accuracy: 21.57%
Epoch [9/10], Training Loss: 2.191, Validation Accuracy: 22.21%
Epoch [10/10], Training Loss: 2.159, Validation Accuracy: 22.84%
Client 3:
Epoch [1/10], Training Loss: 2.292, Validation Accuracy: 18.88%
Epoch [2/10], Training Loss: 2.290, Validation Accuracy: 19.56%
Epoch [3/10], Training Loss: 2.286, Validation Accuracy: 19.42%
Epoch [4/10], Training Loss: 2.281, Validation Accuracy: 19.43%
Epoch [5/10], Training Loss: 2.272, Validation Accuracy: 19.34%
Epoch [6/10], Training Loss: 2.260, Validation Accuracy: 19.54%
Epoch [7/10], Training Loss: 2.241, Validation Accuracy: 20.44%
Epoch [8/10], Training Loss: 2.216, Validation Accuracy: 21.90%
Epoch [9/10], Training Loss: 2.186, Validation Accuracy: 22.51%
Epoch [10/10], Training Loss: 2.153, Validation Accuracy: 22.93%
Client 4:
Epoch [1/10], Training Loss: 2.293, Validation Accuracy: 19.43%
Epoch [2/10], Training Loss: 2.290, Validation Accuracy: 18.83%
Epoch [3/10], Training Loss: 2.287, Validation Accuracy: 18.54%
Epoch [4/10], Training Loss: 2.282, Validation Accuracy: 18.21%
Epoch [5/10], Training Loss: 2.274, Validation Accuracy: 18.36%
Epoch [6/10], Training Loss: 2.263, Validation Accuracy: 18.28%
Epoch [7/10], Training Loss: 2.245, Validation Accuracy: 18.63%
Epoch [8/10], Training Loss: 2.221, Validation Accuracy: 20.34%
Epoch [9/10], Training Loss: 2.191, Validation Accuracy: 21.74%
Epoch [10/10], Training Loss: 2.158, Validation Accuracy: 22.74%
Client 5:
Epoch [1/10], Training Loss: 2.292, Validation Accuracy: 19.56%
Epoch [2/10], Training Loss: 2.290, Validation Accuracy: 19.95%
Epoch [3/10], Training Loss: 2.286, Validation Accuracy: 20.11%
Epoch [4/10], Training Loss: 2.281, Validation Accuracy: 19.80%
Epoch [5/10], Training Loss: 2.273, Validation Accuracy: 20.01%
Epoch [6/10], Training Loss: 2.261, Validation Accuracy: 20.08%
Epoch [7/10], Training Loss: 2.242, Validation Accuracy: 19.90%
Epoch [8/10], Training Loss: 2.215, Validation Accuracy: 20.67%
Epoch [9/10], Training Loss: 2.185, Validation Accuracy: 21.59%
Epoch [10/10], Training Loss: 2.151, Validation Accuracy: 23.08%
--- Round 3 Complete: Global Test Accuracy: 23.06% ---

--- Round 4/10 ---
Client 1:
Epoch [1/10], Training Loss: 2.123, Validation Accuracy: 23.62%
Epoch [2/10], Training Loss: 2.095, Validation Accuracy: 24.08%
Epoch [3/10], Training Loss: 2.074, Validation Accuracy: 24.89%
Epoch [4/10], Training Loss: 2.055, Validation Accuracy: 25.16%
Epoch [5/10], Training Loss: 2.037, Validation Accuracy: 25.46%
Epoch [6/10], Training Loss: 2.019, Validation Accuracy: 26.37%
Epoch [7/10], Training Loss: 2.003, Validation Accuracy: 26.70%
Epoch [8/10], Training Loss: 1.989, Validation Accuracy: 26.71%
Epoch [9/10], Training Loss: 1.975, Validation Accuracy: 27.61%
Epoch [10/10], Training Loss: 1.959, Validation Accuracy: 28.35%
Client 2:
Epoch [1/10], Training Loss: 2.130, Validation Accuracy: 23.02%
Epoch [2/10], Training Loss: 2.106, Validation Accuracy: 23.54%
Epoch [3/10], Training Loss: 2.085, Validation Accuracy: 24.30%
Epoch [4/10], Training Loss: 2.066, Validation Accuracy: 24.99%
Epoch [5/10], Training Loss: 2.048, Validation Accuracy: 25.57%
Epoch [6/10], Training Loss: 2.031, Validation Accuracy: 25.95%
Epoch [7/10], Training Loss: 2.016, Validation Accuracy: 26.64%
Epoch [8/10], Training Loss: 2.000, Validation Accuracy: 26.96%
Epoch [9/10], Training Loss: 1.985, Validation Accuracy: 27.72%
Epoch [10/10], Training Loss: 1.969, Validation Accuracy: 27.77%
Client 3:
Epoch [1/10], Training Loss: 2.128, Validation Accuracy: 23.18%
Epoch [2/10], Training Loss: 2.104, Validation Accuracy: 23.43%
Epoch [3/10], Training Loss: 2.084, Validation Accuracy: 24.32%
Epoch [4/10], Training Loss: 2.063, Validation Accuracy: 24.56%
Epoch [5/10], Training Loss: 2.043, Validation Accuracy: 25.59%
Epoch [6/10], Training Loss: 2.025, Validation Accuracy: 25.91%
Epoch [7/10], Training Loss: 2.006, Validation Accuracy: 26.04%
Epoch [8/10], Training Loss: 1.991, Validation Accuracy: 26.88%
Epoch [9/10], Training Loss: 1.978, Validation Accuracy: 26.82%
Epoch [10/10], Training Loss: 1.965, Validation Accuracy: 28.16%
Client 4:
Epoch [1/10], Training Loss: 2.129, Validation Accuracy: 23.28%
Epoch [2/10], Training Loss: 2.103, Validation Accuracy: 23.73%
Epoch [3/10], Training Loss: 2.080, Validation Accuracy: 24.24%
Epoch [4/10], Training Loss: 2.060, Validation Accuracy: 24.78%
Epoch [5/10], Training Loss: 2.040, Validation Accuracy: 25.62%
Epoch [6/10], Training Loss: 2.022, Validation Accuracy: 26.06%
Epoch [7/10], Training Loss: 2.003, Validation Accuracy: 26.75%
Epoch [8/10], Training Loss: 1.989, Validation Accuracy: 27.31%
Epoch [9/10], Training Loss: 1.976, Validation Accuracy: 27.89%
Epoch [10/10], Training Loss: 1.963, Validation Accuracy: 28.33%
Client 5:
Epoch [1/10], Training Loss: 2.126, Validation Accuracy: 23.25%
Epoch [2/10], Training Loss: 2.101, Validation Accuracy: 23.19%
Epoch [3/10], Training Loss: 2.079, Validation Accuracy: 23.92%
Epoch [4/10], Training Loss: 2.058, Validation Accuracy: 24.87%
Epoch [5/10], Training Loss: 2.039, Validation Accuracy: 25.44%
Epoch [6/10], Training Loss: 2.022, Validation Accuracy: 26.08%
Epoch [7/10], Training Loss: 2.002, Validation Accuracy: 26.21%
Epoch [8/10], Training Loss: 1.987, Validation Accuracy: 26.57%
Epoch [9/10], Training Loss: 1.974, Validation Accuracy: 27.85%
Epoch [10/10], Training Loss: 1.959, Validation Accuracy: 28.01%
--- Round 4 Complete: Global Test Accuracy: 28.61% ---

--- Round 5/10 ---
Client 1:
Epoch [1/10], Training Loss: 1.952, Validation Accuracy: 28.19%
Epoch [2/10], Training Loss: 1.938, Validation Accuracy: 29.54%
Epoch [3/10], Training Loss: 1.926, Validation Accuracy: 29.57%
Epoch [4/10], Training Loss: 1.911, Validation Accuracy: 30.39%
Epoch [5/10], Training Loss: 1.892, Validation Accuracy: 31.49%
Epoch [6/10], Training Loss: 1.876, Validation Accuracy: 31.85%
Epoch [7/10], Training Loss: 1.860, Validation Accuracy: 32.71%
Epoch [8/10], Training Loss: 1.845, Validation Accuracy: 32.01%
Epoch [9/10], Training Loss: 1.828, Validation Accuracy: 33.64%
Epoch [10/10], Training Loss: 1.809, Validation Accuracy: 34.13%
Client 2:
Epoch [1/10], Training Loss: 1.961, Validation Accuracy: 28.58%
Epoch [2/10], Training Loss: 1.948, Validation Accuracy: 28.99%
Epoch [3/10], Training Loss: 1.931, Validation Accuracy: 29.79%
Epoch [4/10], Training Loss: 1.919, Validation Accuracy: 30.80%
Epoch [5/10], Training Loss: 1.904, Validation Accuracy: 30.26%
Epoch [6/10], Training Loss: 1.887, Validation Accuracy: 31.81%
Epoch [7/10], Training Loss: 1.870, Validation Accuracy: 31.65%
Epoch [8/10], Training Loss: 1.855, Validation Accuracy: 32.79%
Epoch [9/10], Training Loss: 1.838, Validation Accuracy: 33.58%
Epoch [10/10], Training Loss: 1.820, Validation Accuracy: 34.34%
Client 3:
Epoch [1/10], Training Loss: 1.960, Validation Accuracy: 28.44%
Epoch [2/10], Training Loss: 1.944, Validation Accuracy: 29.40%
Epoch [3/10], Training Loss: 1.932, Validation Accuracy: 29.91%
Epoch [4/10], Training Loss: 1.916, Validation Accuracy: 30.37%
Epoch [5/10], Training Loss: 1.900, Validation Accuracy: 30.85%
Epoch [6/10], Training Loss: 1.885, Validation Accuracy: 31.37%
Epoch [7/10], Training Loss: 1.872, Validation Accuracy: 31.97%
Epoch [8/10], Training Loss: 1.856, Validation Accuracy: 32.60%
Epoch [9/10], Training Loss: 1.839, Validation Accuracy: 33.57%
Epoch [10/10], Training Loss: 1.820, Validation Accuracy: 34.15%
Client 4:
Epoch [1/10], Training Loss: 1.956, Validation Accuracy: 29.03%
Epoch [2/10], Training Loss: 1.941, Validation Accuracy: 28.88%
Epoch [3/10], Training Loss: 1.926, Validation Accuracy: 29.92%
Epoch [4/10], Training Loss: 1.913, Validation Accuracy: 30.62%
Epoch [5/10], Training Loss: 1.895, Validation Accuracy: 30.73%
Epoch [6/10], Training Loss: 1.879, Validation Accuracy: 30.88%
Epoch [7/10], Training Loss: 1.863, Validation Accuracy: 32.01%
Epoch [8/10], Training Loss: 1.849, Validation Accuracy: 32.33%
Epoch [9/10], Training Loss: 1.831, Validation Accuracy: 33.96%
Epoch [10/10], Training Loss: 1.813, Validation Accuracy: 34.24%
Client 5:
Epoch [1/10], Training Loss: 1.953, Validation Accuracy: 28.39%
Epoch [2/10], Training Loss: 1.940, Validation Accuracy: 28.38%
Epoch [3/10], Training Loss: 1.926, Validation Accuracy: 29.45%
Epoch [4/10], Training Loss: 1.910, Validation Accuracy: 30.56%
Epoch [5/10], Training Loss: 1.893, Validation Accuracy: 30.01%
Epoch [6/10], Training Loss: 1.877, Validation Accuracy: 31.36%
Epoch [7/10], Training Loss: 1.860, Validation Accuracy: 31.56%
Epoch [8/10], Training Loss: 1.847, Validation Accuracy: 32.12%
Epoch [9/10], Training Loss: 1.830, Validation Accuracy: 32.87%
Epoch [10/10], Training Loss: 1.814, Validation Accuracy: 33.01%
--- Round 5 Complete: Global Test Accuracy: 34.16% ---

--- Round 6/10 ---
Client 1:
Epoch [1/10], Training Loss: 1.808, Validation Accuracy: 34.66%
Epoch [2/10], Training Loss: 1.792, Validation Accuracy: 34.89%
Epoch [3/10], Training Loss: 1.775, Validation Accuracy: 35.89%
Epoch [4/10], Training Loss: 1.757, Validation Accuracy: 35.54%
Epoch [5/10], Training Loss: 1.744, Validation Accuracy: 36.32%
Epoch [6/10], Training Loss: 1.726, Validation Accuracy: 37.14%
Epoch [7/10], Training Loss: 1.715, Validation Accuracy: 37.36%
Epoch [8/10], Training Loss: 1.694, Validation Accuracy: 37.57%
Epoch [9/10], Training Loss: 1.686, Validation Accuracy: 38.43%
Epoch [10/10], Training Loss: 1.674, Validation Accuracy: 38.73%
Client 2:
Epoch [1/10], Training Loss: 1.813, Validation Accuracy: 34.93%
Epoch [2/10], Training Loss: 1.795, Validation Accuracy: 34.92%
Epoch [3/10], Training Loss: 1.780, Validation Accuracy: 36.09%
Epoch [4/10], Training Loss: 1.762, Validation Accuracy: 36.21%
Epoch [5/10], Training Loss: 1.747, Validation Accuracy: 36.86%
Epoch [6/10], Training Loss: 1.729, Validation Accuracy: 37.18%
Epoch [7/10], Training Loss: 1.711, Validation Accuracy: 37.88%
Epoch [8/10], Training Loss: 1.697, Validation Accuracy: 37.73%
Epoch [9/10], Training Loss: 1.681, Validation Accuracy: 38.30%
Epoch [10/10], Training Loss: 1.668, Validation Accuracy: 39.00%
Client 3:
Epoch [1/10], Training Loss: 1.817, Validation Accuracy: 34.52%
Epoch [2/10], Training Loss: 1.797, Validation Accuracy: 35.00%
Epoch [3/10], Training Loss: 1.782, Validation Accuracy: 35.12%
Epoch [4/10], Training Loss: 1.768, Validation Accuracy: 36.62%
Epoch [5/10], Training Loss: 1.746, Validation Accuracy: 36.96%
Epoch [6/10], Training Loss: 1.728, Validation Accuracy: 37.51%
Epoch [7/10], Training Loss: 1.714, Validation Accuracy: 37.97%
Epoch [8/10], Training Loss: 1.696, Validation Accuracy: 38.06%
Epoch [9/10], Training Loss: 1.680, Validation Accuracy: 37.72%
Epoch [10/10], Training Loss: 1.667, Validation Accuracy: 38.14%
Client 4:
Epoch [1/10], Training Loss: 1.809, Validation Accuracy: 34.63%
Epoch [2/10], Training Loss: 1.790, Validation Accuracy: 35.34%
Epoch [3/10], Training Loss: 1.774, Validation Accuracy: 35.39%
Epoch [4/10], Training Loss: 1.757, Validation Accuracy: 36.00%
Epoch [5/10], Training Loss: 1.737, Validation Accuracy: 36.19%
Epoch [6/10], Training Loss: 1.720, Validation Accuracy: 37.30%
Epoch [7/10], Training Loss: 1.707, Validation Accuracy: 37.44%
Epoch [8/10], Training Loss: 1.690, Validation Accuracy: 38.29%
Epoch [9/10], Training Loss: 1.678, Validation Accuracy: 38.65%
Epoch [10/10], Training Loss: 1.665, Validation Accuracy: 38.79%
Client 5:
Epoch [1/10], Training Loss: 1.806, Validation Accuracy: 34.19%
Epoch [2/10], Training Loss: 1.789, Validation Accuracy: 34.93%
Epoch [3/10], Training Loss: 1.774, Validation Accuracy: 35.49%
Epoch [4/10], Training Loss: 1.759, Validation Accuracy: 35.67%
Epoch [5/10], Training Loss: 1.742, Validation Accuracy: 35.04%
Epoch [6/10], Training Loss: 1.729, Validation Accuracy: 36.95%
Epoch [7/10], Training Loss: 1.715, Validation Accuracy: 37.32%
Epoch [8/10], Training Loss: 1.700, Validation Accuracy: 37.70%
Epoch [9/10], Training Loss: 1.691, Validation Accuracy: 38.21%
Epoch [10/10], Training Loss: 1.669, Validation Accuracy: 38.35%
--- Round 6 Complete: Global Test Accuracy: 39.65% ---

--- Round 7/10 ---
Client 1:
Epoch [1/10], Training Loss: 1.671, Validation Accuracy: 38.96%
Epoch [2/10], Training Loss: 1.658, Validation Accuracy: 40.15%
Epoch [3/10], Training Loss: 1.640, Validation Accuracy: 39.80%
Epoch [4/10], Training Loss: 1.628, Validation Accuracy: 40.69%
Epoch [5/10], Training Loss: 1.618, Validation Accuracy: 40.96%
Epoch [6/10], Training Loss: 1.606, Validation Accuracy: 41.02%
Epoch [7/10], Training Loss: 1.598, Validation Accuracy: 40.87%
Epoch [8/10], Training Loss: 1.588, Validation Accuracy: 42.08%
Epoch [9/10], Training Loss: 1.577, Validation Accuracy: 41.28%
Epoch [10/10], Training Loss: 1.563, Validation Accuracy: 41.99%
Client 2:
Epoch [1/10], Training Loss: 1.671, Validation Accuracy: 39.45%
Epoch [2/10], Training Loss: 1.659, Validation Accuracy: 39.25%
Epoch [3/10], Training Loss: 1.645, Validation Accuracy: 40.19%
Epoch [4/10], Training Loss: 1.630, Validation Accuracy: 40.87%
Epoch [5/10], Training Loss: 1.615, Validation Accuracy: 40.62%
Epoch [6/10], Training Loss: 1.602, Validation Accuracy: 40.00%
Epoch [7/10], Training Loss: 1.594, Validation Accuracy: 41.25%
Epoch [8/10], Training Loss: 1.582, Validation Accuracy: 41.33%
Epoch [9/10], Training Loss: 1.568, Validation Accuracy: 42.52%
Epoch [10/10], Training Loss: 1.566, Validation Accuracy: 41.41%
Client 3:
Epoch [1/10], Training Loss: 1.679, Validation Accuracy: 39.12%
Epoch [2/10], Training Loss: 1.660, Validation Accuracy: 39.83%
Epoch [3/10], Training Loss: 1.640, Validation Accuracy: 40.06%
Epoch [4/10], Training Loss: 1.630, Validation Accuracy: 40.54%
Epoch [5/10], Training Loss: 1.613, Validation Accuracy: 40.87%
Epoch [6/10], Training Loss: 1.610, Validation Accuracy: 40.59%
Epoch [7/10], Training Loss: 1.585, Validation Accuracy: 41.27%
Epoch [8/10], Training Loss: 1.578, Validation Accuracy: 41.85%
Epoch [9/10], Training Loss: 1.573, Validation Accuracy: 41.40%
Epoch [10/10], Training Loss: 1.557, Validation Accuracy: 42.66%
Client 4:
Epoch [1/10], Training Loss: 1.671, Validation Accuracy: 39.40%
Epoch [2/10], Training Loss: 1.654, Validation Accuracy: 39.69%
Epoch [3/10], Training Loss: 1.640, Validation Accuracy: 39.33%
Epoch [4/10], Training Loss: 1.631, Validation Accuracy: 39.91%
Epoch [5/10], Training Loss: 1.617, Validation Accuracy: 40.70%
Epoch [6/10], Training Loss: 1.603, Validation Accuracy: 40.86%
Epoch [7/10], Training Loss: 1.592, Validation Accuracy: 41.26%
Epoch [8/10], Training Loss: 1.589, Validation Accuracy: 41.29%
Epoch [9/10], Training Loss: 1.569, Validation Accuracy: 41.52%
Epoch [10/10], Training Loss: 1.559, Validation Accuracy: 41.56%
Client 5:
Epoch [1/10], Training Loss: 1.675, Validation Accuracy: 39.49%
Epoch [2/10], Training Loss: 1.661, Validation Accuracy: 38.77%
Epoch [3/10], Training Loss: 1.650, Validation Accuracy: 40.31%
Epoch [4/10], Training Loss: 1.640, Validation Accuracy: 40.25%
Epoch [5/10], Training Loss: 1.622, Validation Accuracy: 40.89%
Epoch [6/10], Training Loss: 1.616, Validation Accuracy: 40.49%
Epoch [7/10], Training Loss: 1.605, Validation Accuracy: 41.52%
Epoch [8/10], Training Loss: 1.595, Validation Accuracy: 41.71%
Epoch [9/10], Training Loss: 1.583, Validation Accuracy: 40.96%
Epoch [10/10], Training Loss: 1.572, Validation Accuracy: 41.83%
--- Round 7 Complete: Global Test Accuracy: 43.06% ---

--- Round 8/10 ---
Client 1:
Epoch [1/10], Training Loss: 1.577, Validation Accuracy: 42.83%
Epoch [2/10], Training Loss: 1.570, Validation Accuracy: 43.51%
Epoch [3/10], Training Loss: 1.556, Validation Accuracy: 42.55%
Epoch [4/10], Training Loss: 1.542, Validation Accuracy: 44.05%
Epoch [5/10], Training Loss: 1.526, Validation Accuracy: 44.08%
Epoch [6/10], Training Loss: 1.519, Validation Accuracy: 43.71%
Epoch [7/10], Training Loss: 1.511, Validation Accuracy: 44.44%
Epoch [8/10], Training Loss: 1.502, Validation Accuracy: 44.02%
Epoch [9/10], Training Loss: 1.490, Validation Accuracy: 45.35%
Epoch [10/10], Training Loss: 1.483, Validation Accuracy: 45.05%
Client 2:
Epoch [1/10], Training Loss: 1.572, Validation Accuracy: 42.75%
Epoch [2/10], Training Loss: 1.559, Validation Accuracy: 43.37%
Epoch [3/10], Training Loss: 1.548, Validation Accuracy: 44.09%
Epoch [4/10], Training Loss: 1.540, Validation Accuracy: 43.65%
Epoch [5/10], Training Loss: 1.529, Validation Accuracy: 44.56%
Epoch [6/10], Training Loss: 1.517, Validation Accuracy: 42.70%
Epoch [7/10], Training Loss: 1.515, Validation Accuracy: 44.30%
Epoch [8/10], Training Loss: 1.504, Validation Accuracy: 45.44%
Epoch [9/10], Training Loss: 1.487, Validation Accuracy: 44.83%
Epoch [10/10], Training Loss: 1.478, Validation Accuracy: 45.73%
Client 3:
Epoch [1/10], Training Loss: 1.572, Validation Accuracy: 42.43%
Epoch [2/10], Training Loss: 1.560, Validation Accuracy: 43.52%
Epoch [3/10], Training Loss: 1.543, Validation Accuracy: 43.09%
Epoch [4/10], Training Loss: 1.535, Validation Accuracy: 43.96%
Epoch [5/10], Training Loss: 1.527, Validation Accuracy: 43.67%
Epoch [6/10], Training Loss: 1.513, Validation Accuracy: 44.10%
Epoch [7/10], Training Loss: 1.503, Validation Accuracy: 45.12%
Epoch [8/10], Training Loss: 1.494, Validation Accuracy: 43.69%
Epoch [9/10], Training Loss: 1.486, Validation Accuracy: 45.78%
Epoch [10/10], Training Loss: 1.470, Validation Accuracy: 45.80%
Client 4:
Epoch [1/10], Training Loss: 1.577, Validation Accuracy: 42.10%
Epoch [2/10], Training Loss: 1.562, Validation Accuracy: 43.50%
Epoch [3/10], Training Loss: 1.552, Validation Accuracy: 43.22%
Epoch [4/10], Training Loss: 1.538, Validation Accuracy: 43.89%
Epoch [5/10], Training Loss: 1.531, Validation Accuracy: 44.72%
Epoch [6/10], Training Loss: 1.516, Validation Accuracy: 44.76%
Epoch [7/10], Training Loss: 1.508, Validation Accuracy: 44.80%
Epoch [8/10], Training Loss: 1.496, Validation Accuracy: 44.57%
Epoch [9/10], Training Loss: 1.490, Validation Accuracy: 45.33%
Epoch [10/10], Training Loss: 1.476, Validation Accuracy: 45.19%
Client 5:
Epoch [1/10], Training Loss: 1.582, Validation Accuracy: 42.34%
Epoch [2/10], Training Loss: 1.570, Validation Accuracy: 43.49%
Epoch [3/10], Training Loss: 1.560, Validation Accuracy: 43.33%
Epoch [4/10], Training Loss: 1.548, Validation Accuracy: 44.23%
Epoch [5/10], Training Loss: 1.538, Validation Accuracy: 44.08%
Epoch [6/10], Training Loss: 1.524, Validation Accuracy: 44.43%
Epoch [7/10], Training Loss: 1.520, Validation Accuracy: 44.52%
Epoch [8/10], Training Loss: 1.509, Validation Accuracy: 44.71%
Epoch [9/10], Training Loss: 1.499, Validation Accuracy: 44.97%
Epoch [10/10], Training Loss: 1.491, Validation Accuracy: 44.29%
--- Round 8 Complete: Global Test Accuracy: 46.08% ---

--- Round 9/10 ---
Client 1:
Epoch [1/10], Training Loss: 1.498, Validation Accuracy: 45.84%
Epoch [2/10], Training Loss: 1.485, Validation Accuracy: 45.95%
Epoch [3/10], Training Loss: 1.479, Validation Accuracy: 46.80%
Epoch [4/10], Training Loss: 1.465, Validation Accuracy: 46.71%
Epoch [5/10], Training Loss: 1.455, Validation Accuracy: 47.10%
Epoch [6/10], Training Loss: 1.446, Validation Accuracy: 46.50%
Epoch [7/10], Training Loss: 1.437, Validation Accuracy: 47.59%
Epoch [8/10], Training Loss: 1.428, Validation Accuracy: 47.32%
Epoch [9/10], Training Loss: 1.417, Validation Accuracy: 47.69%
Epoch [10/10], Training Loss: 1.408, Validation Accuracy: 47.56%
Client 2:
Epoch [1/10], Training Loss: 1.494, Validation Accuracy: 46.22%
Epoch [2/10], Training Loss: 1.488, Validation Accuracy: 46.65%
Epoch [3/10], Training Loss: 1.471, Validation Accuracy: 46.62%
Epoch [4/10], Training Loss: 1.467, Validation Accuracy: 46.09%
Epoch [5/10], Training Loss: 1.448, Validation Accuracy: 46.77%
Epoch [6/10], Training Loss: 1.444, Validation Accuracy: 46.47%
Epoch [7/10], Training Loss: 1.443, Validation Accuracy: 46.46%
Epoch [8/10], Training Loss: 1.429, Validation Accuracy: 47.25%
Epoch [9/10], Training Loss: 1.408, Validation Accuracy: 47.50%
Epoch [10/10], Training Loss: 1.407, Validation Accuracy: 46.64%
Client 3:
Epoch [1/10], Training Loss: 1.490, Validation Accuracy: 46.17%
Epoch [2/10], Training Loss: 1.477, Validation Accuracy: 46.32%
Epoch [3/10], Training Loss: 1.473, Validation Accuracy: 46.29%
Epoch [4/10], Training Loss: 1.462, Validation Accuracy: 46.47%
Epoch [5/10], Training Loss: 1.452, Validation Accuracy: 46.14%
Epoch [6/10], Training Loss: 1.441, Validation Accuracy: 47.05%
Epoch [7/10], Training Loss: 1.427, Validation Accuracy: 47.45%
Epoch [8/10], Training Loss: 1.415, Validation Accuracy: 47.70%
Epoch [9/10], Training Loss: 1.420, Validation Accuracy: 47.62%
Epoch [10/10], Training Loss: 1.407, Validation Accuracy: 47.55%
Client 4:
Epoch [1/10], Training Loss: 1.497, Validation Accuracy: 46.20%
Epoch [2/10], Training Loss: 1.486, Validation Accuracy: 46.44%
Epoch [3/10], Training Loss: 1.473, Validation Accuracy: 46.60%
Epoch [4/10], Training Loss: 1.465, Validation Accuracy: 47.05%
Epoch [5/10], Training Loss: 1.450, Validation Accuracy: 46.82%
Epoch [6/10], Training Loss: 1.442, Validation Accuracy: 46.45%
Epoch [7/10], Training Loss: 1.429, Validation Accuracy: 46.85%
Epoch [8/10], Training Loss: 1.417, Validation Accuracy: 47.21%
Epoch [9/10], Training Loss: 1.415, Validation Accuracy: 47.29%
Epoch [10/10], Training Loss: 1.410, Validation Accuracy: 47.11%
Client 5:
Epoch [1/10], Training Loss: 1.511, Validation Accuracy: 46.54%
Epoch [2/10], Training Loss: 1.493, Validation Accuracy: 46.47%
Epoch [3/10], Training Loss: 1.491, Validation Accuracy: 46.48%
Epoch [4/10], Training Loss: 1.478, Validation Accuracy: 46.45%
Epoch [5/10], Training Loss: 1.469, Validation Accuracy: 45.92%
Epoch [6/10], Training Loss: 1.454, Validation Accuracy: 46.60%
Epoch [7/10], Training Loss: 1.451, Validation Accuracy: 46.42%
Epoch [8/10], Training Loss: 1.439, Validation Accuracy: 47.03%
Epoch [9/10], Training Loss: 1.424, Validation Accuracy: 47.65%
Epoch [10/10], Training Loss: 1.425, Validation Accuracy: 47.90%
--- Round 9 Complete: Global Test Accuracy: 48.74% ---

--- Round 10/10 ---
Client 1:
Epoch [1/10], Training Loss: 1.446, Validation Accuracy: 48.40%
Epoch [2/10], Training Loss: 1.427, Validation Accuracy: 47.94%
Epoch [3/10], Training Loss: 1.415, Validation Accuracy: 48.79%
Epoch [4/10], Training Loss: 1.403, Validation Accuracy: 48.56%
Epoch [5/10], Training Loss: 1.398, Validation Accuracy: 48.84%
Epoch [6/10], Training Loss: 1.393, Validation Accuracy: 49.64%
Epoch [7/10], Training Loss: 1.378, Validation Accuracy: 49.07%
Epoch [8/10], Training Loss: 1.369, Validation Accuracy: 49.57%
Epoch [9/10], Training Loss: 1.371, Validation Accuracy: 48.38%
Epoch [10/10], Training Loss: 1.368, Validation Accuracy: 49.07%
Client 2:
Epoch [1/10], Training Loss: 1.433, Validation Accuracy: 48.45%
Epoch [2/10], Training Loss: 1.426, Validation Accuracy: 47.65%
Epoch [3/10], Training Loss: 1.411, Validation Accuracy: 48.78%
Epoch [4/10], Training Loss: 1.402, Validation Accuracy: 48.92%
Epoch [5/10], Training Loss: 1.396, Validation Accuracy: 48.16%
Epoch [6/10], Training Loss: 1.390, Validation Accuracy: 48.84%
Epoch [7/10], Training Loss: 1.375, Validation Accuracy: 49.58%
Epoch [8/10], Training Loss: 1.370, Validation Accuracy: 49.31%
Epoch [9/10], Training Loss: 1.358, Validation Accuracy: 49.13%
Epoch [10/10], Training Loss: 1.344, Validation Accuracy: 49.58%
Client 3:
Epoch [1/10], Training Loss: 1.438, Validation Accuracy: 47.13%
Epoch [2/10], Training Loss: 1.425, Validation Accuracy: 49.10%
Epoch [3/10], Training Loss: 1.410, Validation Accuracy: 48.74%
Epoch [4/10], Training Loss: 1.399, Validation Accuracy: 48.14%
Epoch [5/10], Training Loss: 1.388, Validation Accuracy: 49.27%
Epoch [6/10], Training Loss: 1.379, Validation Accuracy: 49.14%
Epoch [7/10], Training Loss: 1.366, Validation Accuracy: 48.70%
Epoch [8/10], Training Loss: 1.365, Validation Accuracy: 49.54%
Epoch [9/10], Training Loss: 1.357, Validation Accuracy: 48.12%
Epoch [10/10], Training Loss: 1.352, Validation Accuracy: 49.34%
Client 4:
Epoch [1/10], Training Loss: 1.430, Validation Accuracy: 48.53%
Epoch [2/10], Training Loss: 1.426, Validation Accuracy: 47.83%
Epoch [3/10], Training Loss: 1.427, Validation Accuracy: 47.81%
Epoch [4/10], Training Loss: 1.406, Validation Accuracy: 48.26%
Epoch [5/10], Training Loss: 1.398, Validation Accuracy: 48.63%
Epoch [6/10], Training Loss: 1.388, Validation Accuracy: 48.82%
Epoch [7/10], Training Loss: 1.376, Validation Accuracy: 49.04%
Epoch [8/10], Training Loss: 1.361, Validation Accuracy: 49.44%
Epoch [9/10], Training Loss: 1.362, Validation Accuracy: 48.28%
Epoch [10/10], Training Loss: 1.353, Validation Accuracy: 50.10%
Client 5:
Epoch [1/10], Training Loss: 1.449, Validation Accuracy: 48.37%
Epoch [2/10], Training Loss: 1.434, Validation Accuracy: 48.27%
Epoch [3/10], Training Loss: 1.421, Validation Accuracy: 48.29%
Epoch [4/10], Training Loss: 1.416, Validation Accuracy: 47.08%
Epoch [5/10], Training Loss: 1.411, Validation Accuracy: 47.67%
Epoch [6/10], Training Loss: 1.396, Validation Accuracy: 49.28%
Epoch [7/10], Training Loss: 1.382, Validation Accuracy: 48.92%
Epoch [8/10], Training Loss: 1.377, Validation Accuracy: 49.09%
Epoch [9/10], Training Loss: 1.369, Validation Accuracy: 48.45%
Epoch [10/10], Training Loss: 1.363, Validation Accuracy: 49.78%
--- Round 10 Complete: Global Test Accuracy: 50.50% ---

"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [11.41, 10.1, 9.96, 9.85, 9.85, 9.82, 9.83, 9.83, 9.85, 9.89, 11.49, 11.37, 11.23, 11.5, 12.31, 11.58, 10.71, 10.07, 10.0, 9.93, 11.48, 10.44, 10.58, 10.39, 10.28, 10.47, 11.27, 11.55, 12.25, 12.64, 10.39, 9.95, 9.9, 9.89, 9.87, 9.87, 9.87, 9.87, 9.87, 9.87, 10.89, 10.67, 9.91, 9.85, 9.84, 9.84, 9.84, 9.84, 9.85, 9.85, 12.64, 12.62, 11.74, 12.25, 12.77, 13.15, 13.48, 14.91, 15.73, 16.03, 13.97, 14.24, 12.69, 12.68, 12.96, 13.94, 14.58, 14.44, 15.77, 16.67, 13.78, 13.68, 13.63, 14.11, 14.08, 14.45, 14.52, 14.65, 15.43, 16.25, 12.48, 10.35, 9.97, 9.96, 9.94, 9.98, 10.27, 10.51, 11.47, 12.15, 13.08, 12.04, 10.53, 10.09, 10.01, 10.1, 10.71, 11.71, 13.14, 13.79, 19.21, 19.31, 19.58, 19.79, 19.42, 19.33, 19.4, 20.73, 22.36, 23.26, 20.07, 20.06, 19.94, 20.12, 19.6, 19.73, 20.21, 21.57, 22.21, 22.84, 18.88, 19.56, 19.42, 19.43, 19.34, 19.54, 20.44, 21.9, 22.51, 22.93, 19.43, 18.83, 18.54, 18.21, 18.36, 18.28, 18.63, 20.34, 21.74, 22.74, 19.56, 19.95, 20.11, 19.8, 20.01, 20.08, 19.9